In [ ]:
Reading the file

In [45]:
# import json
# import pandas as pd


# with open('Helsinki_pubs_reviews.json', 'r') as file:
#     data = json.load(file)

# reviews = data.get("reviews")

# print(reviews)


In [19]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
import json

blacklist_words = ['bad', 'good', 'super', 'amaz', 'welcom', 'excel', 'order', 'dont', 'wait', 'bad', 'think', 'rude', 'thank', ]
custom_stop_words = set(blacklist_words) | set(CountVectorizer(stop_words='english').get_stop_words())

reviews_list=[]
with open('Helsinki_pubs_reviews_preprocessed.json','r') as f:
    data = json.load(f)
    for bar, reviews in data['reviews'].items():
        for rev in reviews:
            #print(rev, bar_name)
            reviews_list.append(rev)

min_df=5
max_df=0.9
max_iter=15

vectorizer = CountVectorizer(max_df=max_df, min_df=min_df, stop_words=list(custom_stop_words))
X = vectorizer.fit_transform(reviews_list)



word_counts = X.toarray().sum(axis=0)  # Sum the counts across all documents
feature_names = vectorizer.get_feature_names_out()  # Get the feature names (words)

# Step 4: Create a DataFrame for easy analysis
word_freq_df = pd.DataFrame({'word': feature_names, 'count': word_counts})
word_freq_df = word_freq_df[word_freq_df['count'] > 0]  # Filter out words with zero counts

# Step 5: Sort by count and get the top 20 words
top_words = word_freq_df.sort_values(by='count', ascending=False).head(10)

# Step 6: Extract the list of top words
top_words_list = top_words['word'].tolist()

# Step 7: Update the blacklist by adding top words
blacklist_words = set(blacklist_words).union(top_words_list)




# Step 2: Apply LDA
num_topics = 10  # Adjust this number based on how many topics you expect
lda = LatentDirichletAllocation(n_components=num_topics, max_iter=max_iter, random_state=0)
lda.fit(X)

# Step 3: Get the Topic Distribution for Each Review
topic_distributions = lda.transform(X)

# Step 4: Create a DataFrame to Display Results
topic_df = pd.DataFrame(topic_distributions, columns=[f"Topic_{i+1}" for i in range(num_topics)])
results_df = pd.DataFrame({'review': reviews_list}).join(topic_df)

# Display the results with topic probabilities
#print(results_df.head())

feature_names = vectorizer.get_feature_names_out()
print(feature_names)

# Step 2: Define the number of top words to display for each topic
top_words = 10

# Step 3: Display the top words for each topic
for topic_idx, topic in enumerate(lda.components_):
    #print(f"Topic {topic_idx + 1}:")
    top_features_ind = topic.argsort()[-top_words:][::-1]  # Get indices of the top words
    top_features = [feature_names[i] for i in top_features_ind]
    #print("Top words:", ", ".join(top_features))
    #print()

threshold = 1  # You can adjust this threshold
words_to_remove = []
for idx, word in enumerate(feature_names):
    # Get the weight of the word across all topics
    word_weights = lda.components_[:, idx]
    
    # Count how many topics have a weight above the threshold for this word
    topic_count = np.sum(word_weights > threshold)
    
    # If the word appears in more than two topics, mark it for removal
    if topic_count > 3:
        #blacklist_words.insert(word)
        words_to_remove.append(word)

#black
# print(len(words_to_remove))
# print(words_to_remove)

['abl' 'absolut' 'accept' ... 'youv' 'yummi' 'zero']


In [7]:
print(word_freq_df)

         word  count
0         abl     30
1     absolut    146
2      accept     13
3      access     24
4       accid      7
...       ...    ...
1855  younger     13
1856    youth      9
1857     youv      7
1858    yummi     10
1859     zero      8

[1860 rows x 2 columns]


In [20]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

custom_stop_words = set(blacklist_words) | set(words_to_remove) | set(CountVectorizer(stop_words='english').get_stop_words())

reviews_list=[]
bar_indices=[]
with open('./Helsinki_pubs_reviews_preprocessed.json','r') as f:
    data = json.load(f)
    for bar, reviews in data['reviews'].items():
        for rev in reviews:
            reviews_list.append(rev)
            bar_indices.append(bar)


vectorizer = CountVectorizer(max_df=max_df, min_df=min_df, stop_words=list(custom_stop_words))
X = vectorizer.fit_transform(reviews_list)

# Step 2: Apply LDA
#num_topics = 10  # Adjust this number based on how many topics you expect
lda = LatentDirichletAllocation(n_components=num_topics, max_iter=max_iter, random_state=0)
lda.fit(X)

# Step 3: Get the Topic Distribution for Each Review
topic_distributions = lda.transform(X)

# Step 4: Create a DataFrame to Display Results
# Add columns for each topic probability
topic_df = pd.DataFrame(topic_distributions, columns=[f"Topic_{i+1}" for i in range(num_topics)])
results_df = pd.DataFrame({'review': reviews_list}).join(topic_df)
results_df['id'] = bar_indices

# Display the results with topic probabilities
print(results_df.head())

feature_names = vectorizer.get_feature_names_out()

# Step 2: Define the number of top words to display for each topic
top_words = 10

# Step 3: Display the top words for each topic
for topic_idx, topic in enumerate(lda.components_):
    
    print(f"Topic {topic_idx + 1}:")
    top_features_ind = topic.argsort()[-top_words:][::-1]  # Get indices of the top words
    top_features = [feature_names[i] for i in top_features_ind]
    print("Top words:", ", ".join(top_features))
    #print()



                                              review   Topic_1   Topic_2  \
0   great place individu small group look place m...  0.033340  0.033333   
1   fun place stumbl chanc just look bar shot vod...  0.011111  0.011111   
2   great place quick shot realli nice combin sta...  0.033333  0.033333   
3   spin wheel shot super cool small cozi shot st...  0.014288  0.014288   
4   drink perfect ida amaz art new friend love co...  0.020000  0.020000   

    Topic_3   Topic_4   Topic_5   Topic_6   Topic_7   Topic_8   Topic_9  \
0  0.366670  0.033333  0.033333  0.201696  0.033333  0.033333  0.198293   
1  0.011114  0.011111  0.122223  0.011111  0.376665  0.011111  0.011111   
2  0.033337  0.033333  0.033339  0.033333  0.366652  0.033333  0.033337   
3  0.173978  0.014286  0.014286  0.014286  0.014286  0.140300  0.014289   
4  0.020003  0.290678  0.220000  0.349312  0.020000  0.020000  0.020001   

   Topic_10 id  
0  0.033333  0  
1  0.423331  0  
2  0.366667  0  
3  0.585715  0  
4  0.02

In [56]:
categories = []
barindex = -1
previd = -1
reviewcountforpub = 0
sumofpercentagesforplace = 0
results_df.loc[len(results_df.index)] = [0] * 12
for row, line in results_df.iterrows():
  if line['id'] != previd:
    categories.append([0] * 10)
    if (reviewcountforpub != 0):
      for topicIndex in range(1, 11): # scaling
        categories[barindex][topicIndex-1] /= sumofpercentagesforplace
    barindex += 1
    previd = line['id']
    reviewcountforpub = 0
    sumofpercentagesforplace = 0
  for topicIndex in range(1, 11):
    categories[barindex][topicIndex-1] += line[f'Topic_{topicIndex}']
    sumofpercentagesforplace += line[f'Topic_{topicIndex}']
  reviewcountforpub += 1
print(len(categories))

326


In [57]:
jsonfile = pd.read_json('Helsinki_pubs_reviews_preprocessed.json')
# print(jsonfile[jsonfile['reviews'].apply(len) > 0])
filtered_df = jsonfile[jsonfile['reviews'].apply(len) > 0]
categories.pop()
filtered_df['categories'] = categories
print(filtered_df)
filtered_df.to_json('pubs_and_categories.json')

                                    name  \
0                      Shot Bar Helsinki   
1                       Liberty or Death   
2                               Base Bar   
3                            Bar Chaplin   
4                            Ateljee Bar   
..                                   ...   
332                Corona bar & billiard   
333                                 Ritz   
334                              Baribal   
335  Kamppi Bowling Centre & Biljard Bar   
336                         Biljardi 247   

                                               address  \
0            Yliopistonkatu 8, 00100 Helsinki, Finland   
1             Erottajankatu 5, 00130 Helsinki, Finland   
2               Kalevankatu 3, 00100 Helsinki, Finland   
3           Mannerheimintie 6, 00100 Helsinki, Finland   
4                Yrjönkatu 26, 00100 Helsinki, Finland   
..                                                 ...   
332           Traverssikuja 3, 00510 Helsinki, Finland   
333  Et

/tmp/ipykernel_103113/479401356.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['categories'] = categories


In [58]:
print(filtered_df)

                                    name  \
0                      Shot Bar Helsinki   
1                       Liberty or Death   
2                               Base Bar   
3                            Bar Chaplin   
4                            Ateljee Bar   
..                                   ...   
332                Corona bar & billiard   
333                                 Ritz   
334                              Baribal   
335  Kamppi Bowling Centre & Biljard Bar   
336                         Biljardi 247   

                                               address  \
0            Yliopistonkatu 8, 00100 Helsinki, Finland   
1             Erottajankatu 5, 00130 Helsinki, Finland   
2               Kalevankatu 3, 00100 Helsinki, Finland   
3           Mannerheimintie 6, 00100 Helsinki, Finland   
4                Yrjönkatu 26, 00100 Helsinki, Finland   
..                                                 ...   
332           Traverssikuja 3, 00510 Helsinki, Finland   
333  Et